In [1]:
import sys
import os
from pathlib import Path

# Add src to the module search path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..'))

if src_dir not in sys.path:
    sys.path.append(src_dir)

In [2]:
import pandas as pd
import polars as pl
from src.utils.config_loader import config_loader
from src.data_ingestion.loader import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
ROOTPATH = Path.cwd().parent
RAW_DATA_PATH = ROOTPATH / config_loader.get_data_paths()['raw_data_path']
PROCESSED_DATA_PATH = ROOTPATH / config_loader.get_data_paths()['processed_data_path']

# Dataset Setup: Load, Preprocess, and Save

In [4]:
# Instatiate DataLoader
dataloader = DataLoader()

Run these cells only once

In [5]:
# # Load the raw .csv
# df = dataloader.load_raw_data(RAW_DATA_PATH / 'UserBehavior.csv')

In [6]:
# # Preprocess the raw DataFrame
# df = dataloader.preprocess_data(df)

In [7]:
# # Save preprocessed DataFrame in the format of .parquet
# dataloader.save_preprocessed_parquet(
#     df=df,
#     folder_path=PROCESSED_DATA_PATH,
#     filename='preprocessed.parquet'
# )

In [8]:
df = pl.read_parquet(PROCESSED_DATA_PATH / 'preprocessed.parquet')
# df = pd.read_parquet(PROCESSED_DATA_PATH / 'preprocessed.parquet')

# Exploratory Data Analysis

### (a) Data Inspection

In [ ]:
df.head()

user_id,item_id,category_id,behavior_type,timestamp,datetime,hour,day_of_week,date
i64,i64,i64,str,i64,datetime[ns],i32,i32,date
1,2576651,149192,"""pv""",1511572885,2017-11-25 01:21:25,1,5,2017-11-25
1,3830808,4181361,"""pv""",1511593493,2017-11-25 07:04:53,7,5,2017-11-25
1,4365585,2520377,"""pv""",1511596146,2017-11-25 07:49:06,7,5,2017-11-25
1,4606018,2735466,"""pv""",1511616481,2017-11-25 13:28:01,13,5,2017-11-25
1,230380,411153,"""pv""",1511644942,2017-11-25 21:22:22,21,5,2017-11-25


: 

### (b) Summary Statistics

In [ ]:
df.describe()

### (c) Null Values